In [12]:
from active_critic.utils.gym_utils import sample_expert_transitions, DummyExtractor, make_policy_dict, parse_sampled_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.policies import BaseModel
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import numpy as np
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit
from gym import Env

import gym
import numpy as np

from stable_baselines3 import SAC

In [9]:
seq_len = 100
env_id = 'push'
policy_dict = make_policy_dict()
max_episode_steps = seq_len
env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_id][1]]()
env._freeze_rand_vec = False
env = TimeLimit(env=env, max_episode_steps=max_episode_steps)
env = RolloutInfoWrapper(env)
dv1 = DummyVecEnv([lambda: env])

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, output_size, num_neurons, use_layer_norm=False):
        super(MLP, self).__init__()
        self.use_layer_norm = use_layer_norm

        # create a sequence of linear layers with the specified number of neurons
        self.layers = nn.Sequential(
            *[nn.Linear(input_size if i == 0 else num_neurons[i-1], num_neurons[i]) for i in range(len(num_neurons))]
        )

        # if using layer normalization, create a layer normalization layer for each linear layer
        if self.use_layer_norm:
            self.layer_norms = nn.ModuleList(
                [nn.LayerNorm(num_neurons[i]) for i in range(len(num_neurons))]
            )

        # create the output layer
        self.output_layer = nn.Linear(num_neurons[-1], output_size)

    def forward(self, x):
        # apply the linear layers and optional layer normalization
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if self.use_layer_norm:
                x = self.layer_norms[i](x)
            x = torch.relu(x)

        # apply the output layer and return the result
        return self.output_layer(x)


In [ ]:
env = gym.make("Hopper-v3")

In [ ]:
env.step()

In [ ]:
env.action_space.shape[0]

In [ ]:
class MDPLearner(gym.Wrapper): #Wrapper?
    def __init__(self, env: Env) -> None:
        super().__init__(env)
        self.obsv_predictor = MLP(input_size=env.action_space.shape[0] + env.observation_space.shape[0] , output_size=env.observation_space.shape[0], num_neurons=[64, 64, 64])
        self.rew_predictor = MLP(input_size=env.action_space.shape[0] + env.observation_space.shape[0] , output_size=1, num_neurons=[64, 64, 64])
        self.optimizer = torch.optim.Adam(params=
            [
                {"params": self.obsv_predictor.parameters()},
                {"params": self.rew_predictor.parameters()},
            ]
            , lr = 1e-3
        )

    def predict(self, obsv:th.Tensor, action:th.Tensor, predictor:nn.Module):
        input = th.cat((obsv, action), dim=-1)
        prediction = predictor.forward(input)
        return prediction

    def learn_step(self, obsv1, obsv2, rew1, rew2, action1, action2):
        pred_reward1 = self.predict(obsv=obsv1, action=action1, predictor=self.rew_predictor)
        pred_reward2 = self.predict(obsv=obsv2, action=action2, predictor=self.rew_predictor)

        pred_obsv = self.predict(obsv=obsv1, action=action1, predictor=self.obsv_predictor)

        loss_rew_1 = ((pred_reward1.reshape(-1)-rew1.reshape(-1))**2).mean()
        loss_rew_2 = ((pred_reward2.reshape(-1)-rew2.reshape(-1))**2).mean()

        loss_observation = ((pred_obsv.reshape(-1)-obsv2.reshape(-1))**2).mean()

        loss = loss_rew_1+loss_rew_2+loss_observation

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss_rew_1, loss_rew_2, loss_observation

    def learn(self, obsvs, actions, rewards):
        pass

    def step(self, action):
        next_observation = self.predict(obsv=self.current_obsv, action=action, predictor=self.obsv_predictor)
        next_observation = next_observation.numpy()
        self.current_obsv = next_observation
        self.actions.append(action)
        return next_observation

    def reset(self) -> Any:
        obsv = super().reset()
        self.current_obsv = obsv
        self.actions = []
        return obsv


In [ ]:
import torch as th
a = th.ones([3])
a.numpy()

In [ ]:
class MDPLearner(gym.Wrapper): #Wrapper?
    def __init__(self) -> None:
        super().__init__()
        #obsv space
        #action space

    def emit(self, obsv):
        pass
        #emitter forward

    def predict_emb(self, embedding, action):
        pass
        #predicter forward

    def pred_rew(self, embedding):
        pass
        #reward_predictor forward

    def learn(self, obsvs1, obsvs2, reward1, reward2, actions):
        pass
        #emit(obsv1) -> reward1*
        #emit(obsv1) -> actions -> predictions -> reward2*
        #emit(obsv2) -> reward2**
        #L1 = emit(obsv1) - emit(obsv2)
        #L2 = reward1* - reward1
        #L2 = reward2* - reward2
        #L3 = reward2 ** - reward2

    def step(self, action):
        pass
        #obsv = self.env.step(action)
        #return self.emit(obsv)

    def reset(self):
        pass
        #self.env.reset()

In [ ]:
model_push = SAC("MlpPolicy", env, verbose=1, device='cpu')


In [ ]:
#env = gym.make("Hopper-v3")

model_push = SAC("MlpPolicy", dv1, verbose=1)
model_push.learn(total_timesteps=100*100, log_interval=4)
model_push.save("sac_push")

model = SAC.load("sac_push")

'''while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()'''

In [3]:
model_push = SAC("MlpPolicy", dv1, verbose=1)

Using cuda device


In [ ]:
env = gym.make("Hopper-v2")


In [4]:
obs = dv1.reset()
rews = []
while True:
    action, _states = model_push.predict(obs, deterministic=True)
    obs, reward, done, info = dv1.step(action)
    rews.append(reward)
    #env.render()
    if done:
      break

In [10]:
transitions = sample_expert_transitions(policy=model_push.predict, env=dv1, episodes=10)

Sampling transitions. 10


In [14]:
actions, observations, rewards = parse_sampled_transitions(transitions=transitions, extractor=DummyExtractor(), seq_len=100)

In [16]:
observations.shape

torch.Size([10, 100, 39])

In [ ]:
rews_np = np.array(rews)

In [ ]:
rews_np

In [ ]:
rews_np[-1]

In [ ]:
(rews_np[1:] - rews_np[:-1]).max()

In [ ]:
def make_env(env):
    def _init():
        env._freeze_rand_vec = False
        #rce = ResetCounterWrapper(env)
        riw = RolloutInfoWrapper(env)
        return riw
    return _init

In [ ]:
def make_vec_env_gym(env, num_cpu, seq_len):
    env = SubprocVecEnv([make_env(env) for i in range(num_cpu)])
    return env

In [ ]:
vec_env = make_vec_env_gym(gym.make('Hopper-v3'), 3, 3)

In [ ]:
obsv = vec_env.reset()

In [ ]:
class DummyGymPolicy(BaseModel):
    def __init__(self, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, features_extractor_class: Type[BaseFeaturesExtractor] = ..., features_extractor_kwargs: Optional[Dict[str, Any]] = None, features_extractor: Optional[nn.Module] = None, normalize_images: bool = True, optimizer_class: Type[th.optim.Optimizer] = th.optim.Adam, optimizer_kwargs: Optional[Dict[str, Any]] = None):
        super().__init__(observation_space, action_space, features_extractor_class, features_extractor_kwargs, features_extractor, normalize_images, optimizer_class, optimizer_kwargs)

    def predict(
        self,
        observation: Union[th.Tensor, Dict[str, th.Tensor]],
        state: Optional[Tuple[np.ndarray, ...]] = None,
        episode_start: Optional[np.ndarray] = None,
        deterministic: bool = False,
    ) -> th.Tensor:

        result = np.array([[0,0,0]]*observation.shape[0])
        result[episode_start] = 1
        return result


In [ ]:
epsisode_start = np.array([True, True, False])
dgp = DummyGymPolicy(vec_env.observation_space, vec_env.action_space, features_extractor=DummyExtractor())

In [ ]:
actions = dgp.predict(observation=obsv, episode_start=epsisode_start)

In [ ]:
transitions = sample_expert_transitions(policy=dgp.predict, env=vec_env, episodes=10)

In [ ]:
tf = 0
for i, transition in enumerate(transitions):
    if transition['dones']:
        tf += 1

In [ ]:
ttf = 0
for act in transitions:
    if act['acts'][0] == 1:
        ttf+=1
ttf

In [ ]:
for k in transitions[21]['infos']['rollout']:
    print(k)

In [ ]:
obsv.shape

In [ ]:
from active_critic.model_src.transformer import *
import torch as th

In [ ]:
seq_len = 6
batch_size = 2
dim = 3

tms = ModelSetup()
tms.d_hid = 12
tms.d_model = 12
tms.d_output = 2
tms.device = 'cuda'
tms.nhead = 1
tms.nlayers = 2
tms.seq_len = seq_len

tm = TransformerModel(model_setup=tms)

In [ ]:
inpt = th.ones([batch_size, seq_len, dim], device='cuda')

In [ ]:
result = tm.forward(inpt, offset=0)

In [ ]:
result.shape